<a href="https://colab.research.google.com/github/ShykerBogdan/Prozzoro_project/blob/master/Data_prepr_batches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
import time
from datetime import datetime
import numpy as np

In [4]:
filename='/content/gdrive/My Drive/ML_proz/data_source.csv'
df=pd.read_csv(filename)
df.shape

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,5,14,15,16,17,18,20,24,38,48,50,51,53) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(3507778, 56)

In [1]:
df['tenders.procuringEntity_address_postalCode'] = df['tenders.procuringEntity_address_postalCode'].astype(str).str[:2]

NameError: ignored

In [0]:
class DataPreproccessing:
    def __init__(self, data_source, churn_interval, start_period=None, end_period=None):
        if start_period:
          data_source=data_source[pd.to_datetime(data_source['date_from_id'])>start_period]
        self.last_day = pd.to_datetime('2018-11-29')
        # для статистики постачальника вибираємо проміжок [start_period,end_period]
        self.divide_data = self.last_day - pd.DateOffset(days=churn_interval)
        print(self.divide_data)
        self.data_source = data_source[pd.to_datetime(data_source['date_from_id']) <= self.divide_data]
        # для labeling вибираємо період [end_period, end_period+churn_interval]
        self.label_data = data_source[pd.to_datetime(data_source['date_from_id']) > self.divide_data]
        # вибираємо унікальних постачальників
        self.unique_id = list(self.data_source['participants'].value_counts().index)
        # створюємо dataframe з колонкою унікальних постачальників і туди будемо додавати features
        self.feature_data = pd.DataFrame(self.unique_id, columns=['unique_id'])

    def activity_date(self):
        f = ['winner', 'tenders.auctionPeriod_startDate', 'contracts.dateSigned', 'date_from_id', 'awards.date']
        self.data_source['new_date'] = self.data_source[f].apply(
            lambda x: x[f[2]] if x[f[0] == 1] else (x[f[3]] if pd.isnull(x[f[1]]) else x[f[1]]), axis=1)
        self.data_source['new_date'] = self.data_source[['new_date']].apply(lambda x: x['new_date'][:10], axis=1)
        self.data_source['new_date'] = self.data_source[['new_date', 'date_from_id']].apply(
            lambda x: x['date_from_id'] if x['new_date'] == '\\N' else x['new_date'], axis=1)

    def cleaning(self):
        self.data_source['winner'] = self.data_source[['contracts.value_amount', 'contracts.status', 'winner']].apply \
            (lambda x: 0 if x['contracts.value_amount'] == '\\N'
                            or x['contracts.status'] == 'cancelled' else x['winner'], axis=1)
        print('winner updated')
        self.activity_date()
        print('activity date')

        def convert_to_float(raw):
            try:
                raw['lots.value_amount'] = float(raw['lots.value_amount'])
            except:
                raw['lots.value_amount'] = np.nan
            return raw

        self.data_source['lots.value_amount'] = self.data_source[['lots.value_amount']].apply(convert_to_float, axis=1)
        print('convert lots')
        self.data_source['tenders.procuringEntity_address_postalCode']=self.data_source['tenders.procuringEntity_address_postalCode'].astype(str).str[:2]
        table_postal_code={'Київ':['01', '02', '03', '04', '05', '06'],
                  'Київська область':['07', '08', '09'],
                  'Житомирська область':['10', '11', '12', '13'],
                  'Черніговська область':['14', '15', '16', '17'],
                  'Черкаська область':['18', '19', '20'],
                  'Вінницька область':['21', '22', '23', '24'],
                  'Кіровоградська область':['25', '26', '27', '28'],
                  'Хмельницька область':['29', '30', '31', '32'],
                  'Рівненська область': ['33', '34', '35'],
                  'Полтавська область':['36', '37', '38', '39'],
                  'Сумська область':['40', '41', '42'],
                  'Волинська область':['43', '44', '45'],
                  'Тернопольська область':['46', '47', '48'],
                  'Дніпропетровська область':['49', '50', '51', '52', '53'],
                  'Миколаївська область':['54', '55', '56', '57'],
                  'Чернівецька область':['58', '59', '60'],
                  'Харьківська область':['61', '62', '63', '64'],
                  'Одеська область':['65', '66', '67', '68'],
                  'Запоріжська область':['69', '70', '71', '72'],
                  'Херсонська область':['73', '74', '75'],
                  'Івано-Франківська область':['76', '77', '78'],
                  'Львівська область':['79', '80', '81', '82'],
                  'Донецька область':['83', '84', '85', '86', '87'],
                  'Закарпатська область':['88', '89', '90'],
                  'Луганська область':['91', '92', '93', '94'],
                  'Автономна республіка Крим':['95', '96', '97', '98'],
                  'Севастопіль':['99']}
        regions=dict()
        for key,value in table_postal_code.items():
          for item in value:
            regions[item]=key
        def get_region(raw,regions):
            try:
              raw['region']=regions[raw['tenders.procuringEntity_address_postalCode']]
            except:
              raw['region']=np.nan
        self.data_source['region']=self.data_source[['tenders.procuringEntity_address_postalCode']].apply(get_region,args=(regions,),axis=1)
        print('regions updated')
    def split_into_batches(self, n=20000, filename=None):
        self.cleaning()
        feature_batches = list()

        n_iter = int(np.ceil(self.feature_data.shape[0] / n))
        for i in range(n_iter):
            print('Batch {0}-{1}:'.format(i * n, (i + 1) * n))
            unique_id = self.feature_data.iloc[i * n:(i + 1) * n]['unique_id'].values
            feature_df = pd.DataFrame(unique_id, columns=['unique_id'])
            feature_batches.append(feature_df)
            # part of data_source
            df_ = self.data_source.loc[self.data_source['participants'].isin(unique_id)]
            print('Unique_supp:{0}-{1}'.format(len(unique_id), df_.shape[0]))
            df_ = df_.sort_values(by='new_date')
            self.divide_data = str(self.divide_data)[:10]

            # features:
            # 0. First activity
            # 1. Last activity
            def get_last_activity(raw, data):
                date=data[data['participants'] == raw['unique_id']][['region','new_date']]
                dates=date['new_date'].values
                raw['unique_regions']=len(set(date['region'].values))
                raw['last_activity_date'] = dates[-1]
                try:
                    raw['first_activity_days'] = (
                        pd.to_datetime(self.divide_data) - pd.to_datetime(dates[0])).days
                    raw['last_activity_days'] = (
                        pd.to_datetime(self.divide_data) - pd.to_datetime(raw['last_activity_date'])).days
                    raw['average_activity']=(pd.to_datetime(dates[0])-pd.to_datetime(dates[-1])).days/(len(dates)-1)
                except:
                    print('--------------')
                    raw['last_activity_days'] = '?'
                    raw['first_activity_days'] = '?'
                    raw['average_activity'] = '?'
                    
                return raw

            feature_batches[i] = feature_batches[i].apply(get_last_activity, args=(df_[['participants','new_date','region']],),
                                                          axis=1)
            feature_batches[i]['last_activity_days'] = feature_batches[i].apply(
                lambda x: 0 if x['last_activity_days'] < 0 else x['last_activity_days'], axis=1)

            # 2. Average activity
#             def get_average_activity(raw, data):
#                 dates_ = list(data[data['participants'] == raw['unique_id']]['new_date'].values)
#                 try:
#                     raw['average_activity'] = (pd.to_datetime(dates_[-1]) - pd.to_datetime(dates_[0])).days / len(dates_)
#                 except:
#                     raw['average_activity'] = np.nan
#                 return raw

#             feature_batches[i] = feature_batches[i].apply(get_average_activity, args=(df_[['participants','new_date']],),
#                                                           axis=1)

            # 3. Count lots
            freq = df_['participants'].value_counts()

            def count_lots(raw, frequency):
                raw['count_lots'] = frequency[raw['unique_id']]
                return raw

            feature_batches[i] = feature_batches[i].apply(count_lots, args=(freq,), axis=1)

#             # 4. Win, lose
#             frequency = df_[df_['winner'] == 1]['participants'].value_counts()

#             def win_lose(raw, frequency):
#                 # use try bsc if index not in frequency 'win'=0
#                 try:
#                     raw['win'] = frequency[raw['unique_id']]
#                 except:
#                     raw['win'] = 0
#                 return raw

#             feature_batches[i] = feature_batches[i].apply(win_lose, args=(
#                 frequency,), axis=1)
#             feature_batches[i]['lose'] = feature_batches[i]['count_lots'] - feature_batches[i]['win']
            # 5. Count win open

#             frequency = df_.loc[(df_['winner'] == 1) & (df_['tenders.procurementMethod'] == 'open')][
#                 'participants'].value_counts()

#             def count_win_open(raw, frequency):
#                 try:
#                     raw['win_open'] = frequency[raw['unique_id']]
#                 except:
#                     raw['win_open'] = 0
#                 return raw

#             feature_batches[i] = feature_batches[i].apply(count_win_open, args=(
#                 frequency,), axis=1)
#             feature_batches[i]['win_not_open'] = feature_batches[i]['win'] - feature_batches[i]['win_open']
            # 6. Count lose open
#             frequency = df_.loc[(df_['winner'] == 0) & (df_['tenders.procurementMethod'] == 'open')][
#                 'participants'].value_counts()

#             def count_lose_open(raw, frequency):
#                 try:
#                     raw['lose_open'] = frequency[raw['unique_id']]
#                 except:
#                     raw['lose_open'] = 0
#                 return raw

#             feature_batches[i] = feature_batches[i].apply(count_lose_open, args=(
#                 frequency,), axis=1)
#             feature_batches[i]['lose_not_open'] = feature_batches[i]['lose'] - feature_batches[i]['lose_open']
            # 7. Average economy
            data = df_[df_['winner'] == 1][
                ['participants', 'awards.value_amount', 'lots.value_amount']]
            data['economy_value'] = data['lots.value_amount'] - data['awards.value_amount']
            data['economy_percent'] = data['awards.value_amount'] / data['lots.value_amount']

            def get_economy(raw, data):
                d = data[data['participants'] == raw['unique_id']]
                raw['economy_value'] = np.nanmean(d['economy_value'])
                raw['average_winner_price']=np.nansum(d['awards.value_amount'])/d.shape[0]
                raw['economy_percent'] = np.nanmean(d['economy_percent'])
                return raw

            feature_batches[i] = feature_batches[i].apply(get_economy, args=(data,), axis=1)
#             8. Get label
            participants = list(self.label_data['participants'].value_counts().index)

            def get_y(raw, participants):
              if raw['unique_id'] in participants:
                  raw['y'] = 1
              else:
                  raw['y'] = 0
              return raw
#             9. Get favourite hosts
            feature_batches[i] = feature_batches[i].apply(get_y, args=(participants,), axis=1)
            def favourite_percent(raw, data, eps = 0.7):
                d = data[data['participants']==raw['unique_id']].drop_duplicates('tenders.id')
                try:
                  if d['tenders.procuringEntity_identifier_id'].value_counts().iloc[0]/d.shape[0] >=eps:
                    raw['favourite_host']=1
                  else:
                    raw['favourite_host']=0
                    raw['count_unique_hosts']=d['tenders.procuringEntity_identifier_id'].value_counts().shape[0]
                except:
                  raw['favourite_host']=np.nan
                  raw['count_unique_hosts']=np.nan
                del d
                return raw
            feature_batches[i] = feature_batches[i].apply(favourite_percent, args=(df_[['participants', 'tenders.id', 'tenders.procuringEntity_identifier_id']],0.75,), axis=1)
            # 10. Get unique region
            
        if filename:
            print(f'write to csv')
            feature_batches[i].to_csv(filename + '{0}'.format(i) + '.csv', index=False)

        
        # concat dataframes into one
        print('Concat')
        all_batches_df = pd.concat(feature_batches)
        self.feature_data = all_batches_df.copy()
        if filename:
            print('write to csv_all')
            all_batches_df.to_csv(filename + '_all.csv', index=False)

    def get_label(self):
        participants = list(self.label_data['participants'].value_counts().index)

        def get_y(raw, participants):
            if raw['unique_id'] in participants:
                raw['y'] = 1
            else:
                raw['y'] = 0
            return raw

        self.feature_data = self.feature_data.apply(get_y, args=(participants,), axis=1)


In [0]:
col=['winner','tenders.auctionPeriod_startDate','contracts.dateSigned',
     'awards.date','participants','date_from_id','contracts.value_amount','contracts.status','tenders.id','tenders.procuringEntity_identifier_id','tenders.procuringEntity_address_postalCode','lots.value_amount']

In [19]:
start_time = time.time()
df1=df[col]
prepr = DataPreproccessing(df1.iloc[:10000],108)
prepr.split_into_batches(n=1000)
print("--- %s seconds ---" % (time.time() - start_time))
print(prepr.feature_data.head())

2018-08-13 00:00:00
winner updated
activity date
convert lots
regions updated
Batch 0-1000:
Unique_supp:1000-3074
['2016-03-23' '2016-06-10' '2016-07-07' '2016-07-07' '2016-09-06'
 '2016-09-19' '2016-11-04' '2017-01-27' '2017-03-21' '2017-04-12'
 '2017-06-26' '2017-07-05' '2017-07-05' '2017-07-12' '2017-08-01'
 '2017-09-04' '2017-09-11' '2017-09-28' '2017-10-18' '2017-10-19'
 '2017-10-23' '2017-10-24' '2017-10-30' '2017-11-07' '2017-11-13'
 '2017-11-20' '2017-12-01' '2017-12-04' '2017-12-11' '2017-12-14'
 '2017-12-18' '2017-12-21' '2018-02-01' '2018-02-16' '2018-02-28'
 '2018-03-01' '2018-03-13' '2018-03-28' '2018-04-06' '2018-04-11'
 '2018-04-13' '2018-04-18' '2018-04-20' '2018-04-23' '2018-04-26'
 '2018-05-14' '2018-05-15' '2018-05-15' '2018-05-25' '2018-05-29'
 '2018-06-15' '2018-06-18' '2018-06-25' '2018-07-03' '2018-07-04'
 '2018-07-31']
['2016-03-23' '2016-06-10' '2016-07-07' '2016-07-07' '2016-09-06'
 '2016-09-19' '2016-11-04' '2017-01-27' '2017-03-21' '2017-04-12'
 '2017-06-26'

TypeError: ignored

In [0]:
table_postal_code={'Київ':['01', '02', '03', '04', '05', '06'],
                  'Київська область':['07', '08', '09'],
                  'Житомирська область':['10', '11', '12', '13'],
                  'Черніговська область':['14', '15', '16', '17'],
                  'Черкаська область':['18', '19', '20'],
                  'Вінницька область':['21', '22', '23', '24'],
                  'Кіровоградська область':['25', '26', '27', '28'],
                  'Хмельницька область':['29', '30', '31', '32'],
                  'Рівненська область': ['33', '34', '35'],
                  'Полтавська область':['36', '37', '38', '39'],
                  'Сумська область':['40', '41', '42'],
                  'Волинська область':['43', '44', '45'],
                  'Тернопольська область':['46', '47', '48'],
                  'Дніпропетровська область':['49', '50', '51', '52', '53'],
                  'Миколаївська область':['54', '55', '56', '57'],
                  'Чернівецька область':['58', '59', '60'],
                  'Харьківська область':['61', '62', '63', '64'],
                  'Одеська область':['65', '66', '67', '68'],
                  'Запоріжська область':['69', '70', '71', '72'],
                  'Херсонська область':['73', '74', '75'],
                  'Івано-Франківська область':['76', '77', '78'],
                  'Львівська область':['79', '80', '81', '82'],
                  'Донецька область':['83', '84', '85', '86', '87'],
                  'Закарпатська область':['88', '89', '90'],
                  'Луганська область':['91', '92', '93', '94'],
                  'Автономна республіка Крим':['95', '96', '97', '98'],
                  'Севастопіль':['99']}

In [7]:
regions=dict()
for key,value in table_postal_code.items():
  for item in value:
    regions[item]=key
print(regions)

{'01': 'Київ', '02': 'Київ', '03': 'Київ', '04': 'Київ', '05': 'Київ', '06': 'Київ', '07': 'Київська область', '08': 'Київська область', '09': 'Київська область', '10': 'Житомирська область', '11': 'Житомирська область', '12': 'Житомирська область', '13': 'Житомирська область', '14': 'Черніговська область', '15': 'Черніговська область', '16': 'Черніговська область', '17': 'Черніговська область', '18': 'Черкаська область', '19': 'Черкаська область', '20': 'Черкаська область', '21': 'Вінницька область', '22': 'Вінницька область', '23': 'Вінницька область', '24': 'Вінницька область', '25': 'Кіровоградська область', '26': 'Кіровоградська область', '27': 'Кіровоградська область', '28': 'Кіровоградська область', '29': 'Хмельницька область', '30': 'Хмельницька область', '31': 'Хмельницька область', '32': 'Хмельницька область', '33': 'Рівненська область', '34': 'Рівненська область', '35': 'Рівненська область', '36': 'Полтавська область', '37': 'Полтавська область', '38': 'Полтавська область', 